In [ ]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import concurrent.futures

ing=pd.read_csv("/Users/osman/Desktop/Master Project/data/2/recipe40k.csv")
#SOME PREPROCESSING ARE ONGOING:

#delete recipes that have more than one URL
ing = ing.groupby("recipe").filter(lambda x: len(x["URL"].unique()) == 1)

#the data was wrong in respect of black bean. It turned out that, almost all black peppers in can unit were actually black beans. 
#same story for green been.
ing.loc[(ing.ingredientName == 'black pepper') & (ing.ingredientUnit == 'can'), 'ingredientName'] = 'black bean'
ing.loc[(ing.ingredientName == 'green pepper') & (ing.ingredientUnit == 'can'), 'ingredientName'] = 'green bean'
ing.loc[(ing.ingredientName == 'onion') & (ing.ingredientUnit == 'bunch'), 'ingredientName'] = 'green onion'


In [ ]:
#packcanjar FUNCTION
#for all "can", "package" and "jar" ingredient units, it goes to corresponding URL and scrape the ounce version of the ingredient.
#In summary, all the ingredients in "can", "package" and "jar"  units are converted to "ounce" unit.



#18k rows of data includes "can", "package" and "jar" units:
package=ing[(ing.ingredientUnit=="package") |  (ing.ingredientUnit=="can")|(ing.ingredientUnit=="jar")].reset_index(drop=True)

# Function to scrape ingredient quantities from a single URL
def scrape_single_url(url, ingredient_name):
    data = []

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find all ingredient items on the page
    ingredients = soup.find_all("span", {"data-ingredient-name": True})

    for ingredient in ingredients:
        ingredient_text = ingredient.get_text("data-ingredient-name")

        # Check if "package" is in the previous span element with "data-ingredient-unit" attribute
        previous_span = ingredient.find_previous("span", {"data-ingredient-unit": True})
        if previous_span:
            previous_unit_text = previous_span.get_text()
        else:
            previous_unit_text = ""


        if ingredient_name in ingredient_text and "can" in previous_unit_text:
            # Find the quantity associated with the ingredient (if available)
            quantity_span = ingredient.find_previous("span", {"data-ingredient-unit": True})
            if quantity_span:
                quantity = re.search(r'\d+(\.\d+)?', quantity_span.get_text()).group() if quantity_span else "N/A"
            else:
                quantity = "N/A"
            data.append([url, ingredient_name, quantity])
        elif ingredient_name in ingredient_text and "package" in previous_unit_text:
            # Find the quantity associated with the ingredient (if available)
            quantity_span = ingredient.find_previous("span", {"data-ingredient-unit": True})
            if quantity_span:
                quantity = re.search(r'\d+(\.\d+)?', quantity_span.get_text()).group() if quantity_span else "N/A"
            else:
                quantity = "N/A"
            data.append([url, ingredient_name, quantity])
        
        elif ingredient_name in ingredient_text and "jar" in previous_unit_text:
            # Find the quantity associated with the ingredient (if available)
            quantity_span = ingredient.find_previous("span", {"data-ingredient-unit": True})
            if quantity_span:
                quantity = re.search(r'\d+(\.\d+)?', quantity_span.get_text()).group() if quantity_span else "N/A"
            else:
                quantity = "N/A"
            data.append([url, ingredient_name, quantity])
        
        else:
            quantity = "N/A"  # Set a default value for cases where the unit or ingredient_name is not found
        data.append([url, ingredient_name, quantity])

    if not data:
        data = [[url, ingredient_name, "N/A"]]  # Add a default entry if no matches found

    return data

# Function to scrape ingredient quantities from multiple recipe URLs in parallel
def scrape_ingredient_quantities_parallel(df, max_workers=16):
    data = []
    urls = df['URL']
    ingredient_names = df['ingredientName']

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(scrape_single_url, urls, ingredient_names))

    for result in results:
        data.extend(result)

    # Create a DataFrame from the list of data
    df = pd.DataFrame(data, columns=['URL', 'ingredientName', 'ingredientQuantity'])
    #convert to float
    df.ingredientQuantity=df.ingredientQuantity.apply(convert_to_float)
    
    #same ingredients are summed 
    df=df.groupby(["URL","ingredientName"])['ingredientQuantity'].sum().reset_index()
    return df


packcanjar_oz = scrape_ingredient_quantities_parallel(package)
packcanjar_oz


In [ ]:
packcanjar_oz.to_csv("/Users/osman/Desktop/Master Project/data/2/packcanjar_oz.csv",index=False)